In [1]:
#Importing the libraries and packages
import pandas as pd
import csv
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import sklearn
import re #Regular expression operations

In [2]:
#Importing the song lyrics of artists and combining them
df_ar = pd.read_csv("C:/Users/Lenovo/Downloads/archive/csv/ArianaGrande.csv")
df_be = pd.read_csv("C:/Users/Lenovo/Downloads/archive/csv/Beyonce.csv")
df_bi = pd.read_csv("C:/Users/Lenovo/Downloads/archive/csv/BillieEilish.csv")
df_ca = pd.read_csv("C:/Users/Lenovo/Downloads/archive/csv/CardiB.csv")
df_ch = pd.read_csv("C:/Users/Lenovo/Downloads/archive/csv/CharliePuth.csv")
df_co = pd.read_csv("C:/Users/Lenovo/Downloads/archive/csv/ColdPlay.csv")
df_dr = pd.read_csv("C:/Users/Lenovo/Downloads/archive/csv/Drake.csv")
df_du = pd.read_csv("C:/Users/Lenovo/Downloads/archive/csv/DuaLipa.csv")
df_ed = pd.read_csv("C:/Users/Lenovo/Downloads/archive/csv/EdSheeran.csv")
df_em = pd.read_csv("C:/Users/Lenovo/Downloads/archive/csv/Eminem.csv")
df_ju = pd.read_csv("C:/Users/Lenovo/Downloads/archive/csv/JustinBieber.csv")
df_ka = pd.read_csv("C:/Users/Lenovo/Downloads/archive/csv/KatyPerry.csv")
df_kh = pd.read_csv("C:/Users/Lenovo/Downloads/archive/csv/Khalid.csv")
df_la = pd.read_csv("C:/Users/Lenovo/Downloads/archive/csv/LadyGaga.csv")
df_ma = pd.read_csv("C:/Users/Lenovo/Downloads/archive/csv/Maroon5.csv")
df_ni = pd.read_csv("C:/Users/Lenovo/Downloads/archive/csv/NickiMinaj.csv")
df_po = pd.read_csv("C:/Users/Lenovo/Downloads/archive/csv/PostMalone.csv")
df_ri = pd.read_csv("C:/Users/Lenovo/Downloads/archive/csv/Rihanna.csv")
df_se = pd.read_csv("C:/Users/Lenovo/Downloads/archive/csv/SelenaGomez.csv")
df_ta = pd.read_csv("C:/Users/Lenovo/Downloads/archive/csv/TaylorSwift.csv")
df_all = [df_ar, df_be, df_bi, df_ca, df_ch, df_ch, df_co, df_dr, df_du, df_ed, df_em, df_ju, df_ka, 
          df_kh, df_la, df_ma, df_ni, df_po, df_ri, df_se, df_ta] 
df = pd.concat(df_all) #Making the combined dataset a pandas dataframe
df.head()

,Artist,Title,Album,Date,Lyric,Year,Unnamed: 0
0,Ariana Grande,"​thank u, next","thank u, next",2018-11-03,thought i'd end up with sean but he wasn't a m...,2018,NaN
1,Ariana Grande,7 rings,"thank u, next",2019-01-18,yeah breakfast at tiffany's and bottles of bub...,2019,NaN
2,Ariana Grande,​God is a woman,Sweetener,2018-07-13,you you love it how i move you you love it how...,2018,NaN
3,Ariana Grande,Side To Side,Dangerous Woman,2016-05-20,ariana grande nicki minaj i've been here all ...,2016,NaN
4,Ariana Grande,​​no tears left to cry,Sweetener,2018-04-20,right now i'm in a state of mind i wanna be in...,2018,NaN


In [3]:
#Cleaning and organizing the dataset
del df["Unnamed: 0"]
del df["Year"]
df.drop(df.loc[df['Album'] == "Unreleased Songs"].index, inplace=True)
df = df.dropna()

In [4]:
df.groupby("Artist")["Title"].count() #Info about the artists and songs

Artist
Ariana Grande     53
Beyoncé           62
Billie Eilish     42
Cardi B           26
Charlie Puth      54
Coldplay          61
Drake             73
Dua Lipa          45
Ed Sheeran        48
Eminem            77
Justin Bieber     54
Katy Perry        52
Khalid            32
Lady Gaga         59
Maroon 5          43
Nicki Minaj       47
Post Malone       42
Rihanna           92
Selena Gomez      44
Taylor Swift     106
Name: Title, dtype: int64

In [5]:
#Getting the sentiment score using VADER module
analyzer = SentimentIntensityAnalyzer()
df['compound'] = [analyzer.polarity_scores(x)['compound'] for x in df['Lyric']]
df['neg'] = [analyzer.polarity_scores(x)['neg'] for x in df['Lyric']]
df['neu'] = [analyzer.polarity_scores(x)['neu'] for x in df['Lyric']]
df['pos'] = [analyzer.polarity_scores(x)['pos'] for x in df['Lyric']]

In [6]:
df.head()

,Artist,Title,Album,Date,Lyric,compound,neg,neu,pos
0,Ariana Grande,"​thank u, next","thank u, next",2018-11-03,thought i'd end up with sean but he wasn't a m...,0.9998,0.061,0.517,0.422
1,Ariana Grande,7 rings,"thank u, next",2019-01-18,yeah breakfast at tiffany's and bottles of bub...,0.9983,0.058,0.686,0.256
2,Ariana Grande,​God is a woman,Sweetener,2018-07-13,you you love it how i move you you love it how...,0.9982,0.010,0.759,0.231
3,Ariana Grande,Side To Side,Dangerous Woman,2016-05-20,ariana grande nicki minaj i've been here all ...,-0.6497,0.067,0.868,0.065
4,Ariana Grande,​​no tears left to cry,Sweetener,2018-04-20,right now i'm in a state of mind i wanna be in...,0.7234,0.113,0.748,0.139


In [7]:
#Creating a function to decide on the sentiment of the songs by using the compound score interval for VADER
def categorise(row):  
    if row['compound'] >= 0.05:
        return 'Positive'
    elif row['compound'] > -0.05 and row['compound'] < 0.05:
        return 'Neutral'
    elif row['compound'] <= -0.05:
        return 'Negative'

In [8]:
#Applying the function to the dataframe and creating a column for the sentiment decided on using VADER module
df['VADER sentiment'] = df.apply(lambda row: categorise(row), axis=1)
df.sample(5)

,Artist,Title,Album,Date,Lyric,compound,neg,neu,pos,VADER sentiment
21,Billie Eilish,​my boy,dont smile at me,2017-07-28,three and four and babada babadabada mmm mmm ...,0.9992,0.143,0.448,0.409,Positive
7,Coldplay,Adventure of a Lifetime,A Head Full of Dreams,2015-11-06,indecipherable turn your magic on umi she'd ...,0.9981,0.060,0.601,0.339,Positive
28,Katy Perry,Small Talk,Smile (Fan Edition),2019-08-09,isn't it strange that you used to know me all ...,0.9911,0.074,0.760,0.166,Positive
2,Eminem,Godzilla,Music to Be Murdered By,2020-01-17,ugh you're a monster eminem i can swallow a ...,-0.9973,0.165,0.712,0.123,Negative
337,Beyoncé,Irreplaceable (Beyoncé Experience Live),The Beyoncé Experience Live,2007-11-16,to the left to the left to the left to the lef...,0.8750,0.022,0.927,0.051,Positive


In [9]:
len(df[df['VADER sentiment'] == "Positive"]) #Number of positive songs

758

In [10]:
len(df[df['VADER sentiment'] == "Negative"]) #Number of negative songs

344

In [11]:
len(df[df['VADER sentiment'] == "Neutral"]) #Number of neutral songs

10

In [12]:
#Dropping the neutral labeled songs from the dataset
df = df[df['VADER sentiment'] != 'Neutral']
df.to_csv(r'C:\Users\Lenovo\Desktop\bitirme\data\vader_dataset.csv', index=False)